### 1. 패키지 설치

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=128 # k=100이면 제대로 안나옴
)

# 단일 텍스트 파일 로드 및 분할
file_path = './artworks.txt'

loader = TextLoader(file_path, encoding='utf-8')
documents = loader.load_and_split(text_splitter)

# 분할된 문서 수 출력
print(f"총 분할된 문서 수: {len(documents)}")

총 분할된 문서 수: 24464


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

# 올바른 Hugging Face 모델을 사용한 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

# 확인
print("HuggingFaceEmbeddings initialized successfully!")


/tmp/ipykernel_754/746791366.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
/home/chae/ch/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


HuggingFaceEmbeddings initialized successfully!


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 3. Chroma 데이터베이스 초기화 및 문서 추가
collection_name = 'chroma_art2'

database = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory='./chroma_huggingface_1104'
)

print("Chroma database initialized successfully!")

Chroma database initialized successfully!


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [5]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = 'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda"  # CUDA에서 자동 배치
)


The repository for LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 7/7 [00:22<00:00,  3.26s/it]


In [6]:
# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.7,      # 다양성 증가
    top_k=50,             # 상위 k개 토큰 중에서 샘플링
    repetition_penalty=1.03
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_754/3581699272.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [31]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser # module path 수정
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
문서(context)에 있는 정보만 사용해 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 대답하세요.


<|user|>
{context}

Question: {question}


'''
prompt = ChatPromptTemplate.from_template(template)

In [32]:
retriever = database.as_retriever(search_kwargs={"k": 5})

In [33]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

# 체인 구성 수정
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [36]:
query = "김기승의 '낙양성동도이화'에 대해 마크다운으로 알려주세요."

In [37]:
# 마지막에 원하는 형식으로 출력 필터링
response = chain.invoke(query)
print(response)

Human: 
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
문서(context)에 있는 정보만 사용해 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 대답하세요.


<|user|>
[Document(metadata={'source': './artworks.txt'}, page_content='김기승 작가의 1963년작 낙양성동도이화(6곡병)에 대한 작품 개요입니다. 작가명은 김기승, 작품명은 낙양성동도이화(6곡병), 제작연도는 1963년입니다. 재료는 종이에 먹; 6폭 병풍를 사용했으며 규격은 128×31.5×(6)이고, 서예 부문의 작품입니다. 작품번호는 1364입니다. 김기승 작가의 1963년작 낙양성동도이화(6곡병)에 대한 세부 설명은 다음과 같습니다: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기'), Document(metadata={'source': './artworks.txt'}, page_content='김기승 작가의 연도미상년작 자두연두기두재부중읍 칠보시 묵영에 대한 작품 개요입니다. 작가명은 김기승, 작품명은 자두연두기두재부중읍 칠보시 묵영, 제작연도는 연도미상년입니다. 재료는 종이에 먹를 사용했으며 규격은 244×53이고, 서예 부문의 작품입니다. 작품번호는 1297입니다. 김기승 작가의 연도미상년작 자두연두기두재부중읍 칠보시 묵영에 대한 세부 설명은 다음과 같습니다: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가

In [12]:
import re

def extract_answer(text):
    # 정규 표현식을 사용하여 'Answer:' 이후의 모든 텍스트 추출
    match = re.search(r"Answer:\s*(.*)", text, re.DOTALL)
    if match:
        answer = match.group(1).strip()  # 불필요한 공백 제거
    else:
        answer = "답변을 찾을 수 없습니다."
    
    return answer

In [13]:
answer = extract_answer(response)
print(answer)

```markdown
## 김기승 작가의 낙양성동도이화 작품 개요

- **작가명**: 김기승
- **작품명**: 낙양성동도이화
- **제작연도**: 연도미상년
- **재료**: 종이에 먹
- **규격**: 137×58.5 (세로)
- **작품번호**: 1287

### 세부 설명

원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다.

1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다. 김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기 위하여 온몸을 바친다"고 언급하였다.

낙양성동도이화는 김기승의 대표작 중 하나로, 그의 독창적인 서예 스타일과 깊은 한국적 정서가 담긴 작품이다.
```

이 설명은 김기승 작가의 '낙양성동도이화'에 대한 주요 정보와 세부 내용을 포함하고 있습니다.


In [14]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 유사도 점수를 기준으로 내림차순 정렬
docs_and_scores = sorted(docs_and_scores, key=lambda x: x[1], reverse=True)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: ./artworks.txt
  유사도 점수: 0.3268
  전체 내용: 김기승 작가의 연도미상년작 송라예영 중계화혼(행서)에 대한 작품 개요입니다. 작가명은 김기승, 작품명은 송라예영 중계화혼(행서), 제작연도는 연도미상년입니다. 재료는 종이에 먹를 사용했으며 규격은 135×32.5이고, 서예 부문의 작품입니다. 작품번호는 1277입니다. 김기승 작가의 연도미상년작 송라예영 중계화혼(행서)에 대한 세부 설명은 다음과 같습니다: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기 위하여 온몸을

문서 2:
  파일명: ./artworks.txt
  유사도 점수: 0.3264
  전체 내용: 김기승 작가의 1971년작 지립청상송상조명고황권성현론(사명대사 시)(2곡병)에 대한 작품 개요입니다. 작가명은 김기승, 작품명은 지립청상송상조명고황권성현론(사명대사 시)(2곡병), 제작연도는 1971년입니다. 재료는 종이에 먹; 2폭 병풍를 사용했으며 규격은 127×32.5×(2)이고, 서예 부문의 작품입니다. 작품번호는 1349입니다. 김기승 작가의 1971년작 지립청상송상조명고황권성현론(사명대사 시)(2곡병)에 대한 세부 설명은 다음과 같습니다: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955

In [44]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 0번 샘플만 출력
if docs_and_scores:
    doc, score = docs_and_scores[4]
    print("4번 샘플:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")
else:
    print("검색된 문서가 없습니다.")


4번 샘플:
  파일명: ./artworks.txt
  유사도 점수: 0.3220
  전체 내용: 김기승 작가의 연도미상년작 행서(송풍각시 중에서)황산곡체에 대한 작품 개요입니다. 작가명은 김기승, 작품명은 행서(송풍각시 중에서)황산곡체, 제작연도는 연도미상년입니다. 재료는 종이에 먹; 족자를 사용했으며 규격은 374.5×61이고, 서예 부문의 작품입니다. 작품번호는 1196입니다. 김기승 작가의 연도미상년작 행서(송풍각시 중에서)황산곡체에 대한 세부 설명은 다음과 같습니다: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을
